# Project: Building a Database for crime reports

In this project I will build a database for storing data of crimes committed in Boston with Postgres. The dataset is available in the file boston.csv

Here are a description of some of the columns

**incident_number**- represents the identifier of the crime  
**offense_code**- numeric identifier code for the committed crime  
**description**- a description of the crime  
**date**- date crime occurred  
**day_of_the_week**- Day of the week crime occurred  
**lat**- latitude coordination where crime occurred  
**long**- longitude coordination where crime occurred  

Goal of this project is to create a database that is named *crimes_db* with a table- *boston_crimes* - that stores the data from the boston.csv file. The *boston_crimes* table will be contained inside of a schema that is named crimes. 

I will also create two groups that can access the database; one readonly group and one readwrite group. The readonly group represents Data Analysts who would use this database to Select files to do their analysis. The readwrite group represents Data Scientists who would Select and Edit Files but not able to delete tables. Finally, I will create one and add a user for each of the groups.



## Creating the crime_db DataBase

I will start by creating a database for storing the crime data as well as a schema that contains the tables. When connecting to the Postgres server a database name needs to be specified. Because the crime_db database does not exist yet I create it by connecting to a database named "dq" using the username "dq" (short for DataQuest, because this is a Dataquest project)

In [1]:
#Importing PostgreSQL Database Adapter psycopg2
import psycopg2
#Connecting to dq database
conn1 = psycopg2.connect(dbname="dq", user="dq")
cur1 = conn1.cursor()
#Setting conn.autocommit to true to ensure database gets created
conn1.autocommit = True
#Creating new database crime_db
cur1.execute("CREATE DATABASE crime_db;")
conn1.autocommit = False
#Disconnect from "dq" database, going to use the newly created database
conn1.close()

Once I have created the crime_db database from the dq database, I can disconnect from the dq database and exclusively operate from the newly created crime_db database connection.  

Within this newly created database I'm going to create a schema called crimes to store the data.

In [2]:
#Connecting to crime_db database
conn2 = psycopg2.connect(dbname="crime_db", user="dq")
conn2.autocommit = False
cur2 = conn2.cursor()
#Creating schema named crimes
cur2.execute("CREATE SCHEMA crimes;")
#Committing Changes
conn2.commit

<function connection.commit>

## Reading in the DataSet

Now that I have the database and schema I am ready to start creating tables with data. However, before doing so, I need to gather information about the crime dataset that I'm using to ensure the right datatypes are used in the data.

I'll start by reading in all of the column names and printing the first row

In [3]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

To identify the proper datatypes for the columns of my crime data table, I'll first create a function *get_col_set()* (with inputs of a CSV file and a column index) that computes a Python set and returns all of the distinct values of the column.  

This function will be useful for checking whether an enumerated datatype is a good choice to represent a column and for computing the max-length of a text column to get the appropriate size for VARCHAR columns.

In [4]:
#Creating get_col_set() function
def get_col_set(csv_filename, col_index):
    value_set = set() 
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            column = row[col_index]
            if column not in value_set:
                value_set.add(column)
    return value_set

Now I will use the function to compute the number of different values each column contains.

In [5]:
for i in range(len(col_headers)):
    values = get_col_set('boston.csv', i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


Now that I have the number of distinct values for each column I can tell that the description and day_of_the_week columns are textual because of the low number of distinct values.

## Finding Max Length of Description column

In [6]:
#Finding which index the description column is
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [7]:
descriptions = get_col_set('boston.csv', 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


The maximum length for a value in the description column is 58.

## Creating the Table

Now I am finally ready to create the table. The table will be named boston_crimes and will be stored within the crimes schema of the crime_db database.

In [8]:
#Identifying column names by re-printing the headers
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [9]:
#Remembering the datatypes of each column
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


The datatypes of each column are:  

integer  
integer   
string  
date  
string  
decimal  
decimal

Because the day_of_week column has a small range of values it is suitable to have an enumerated datatype so I will create it before making the table.

From the last exercise I know that the maximum length for a value in the description column is 58. So I will set the column type as varchar(100) to be safe.

In [10]:
#Create enumerated datatype for day_of_week column
cur2.execute("CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")
conn2.commit()
#Creating the table
cur2.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description varchar(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
        );
""")

conn2.commit()

## Loading Data into the Table

Now that I have created the table it is now time to load the boston crime data into it. I will be using the cursor.copy_expert( ) method.

In [11]:
cur2.execute("rollback;")
conn2.commit()

In [12]:
cur2.execute("rollback;")
#Opening the csv with the crime data
with open('boston.csv') as f:
#Loading in the data from the csv into our table
    cur2.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    conn2.commit()

## Revoking Public Privileges

Now that I have created the table it is time to handle the users of the table. As stated previously, the goal of this project is to create two groups: readonly and readwrite. The workflow for managing user principles is to first revoke all privileges and then grant the neccesary ones for each group.  

I will begin by revoking all user priviliges. The first step is to make sure there are no inherited privileges from the public group on the public schema.

In [13]:
cur2.execute('REVOKE ALL ON SCHEMA public FROM public;')
cur2.execute('REVOKE ALL ON DATABASE crime_db FROM public;')
conn2.commit()

## Creating User Groups

After I have succesfully ensured that users will not inadvertently inheirt privileges from the public group it is time to create the two user groups.  

As previously mentioned, the two user groups will be readonly and readwrite. The readonly group only has privileges to perform SELECT queries, whereas the readwrite group can perform SELECT, INSERT, DELETE and UPDATE queries. 

In [14]:
#Create readonly group
cur2.execute('CREATE GROUP readonly NOLOGIN;')
#Create readwrite group
cur2.execute('CREATE GROUP readwrite NOLOGIN;')
#Granting connection priviliges to the user groups
cur2.execute('GRANT CONNECT ON DATABASE crime_db TO readonly;')
cur2.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite;')
#Granting usage of the crimes schema to the groups
cur2.execute('GRANT USAGE ON SCHEMA crimes TO readonly;')
cur2.execute('GRANT USAGE ON SCHEMA crimes TO readwrite;')
#Granting SELECT privileges to readonly group
cur2.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;')
cur2.execute('GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;')

## Creating Users

I am almost done with setting up the crime database. I have already created the schema, the table and the user groups. The only thing left to do is to create and add users to the groups. 

I will add two users, one will be a data analyst in the readonly group and one will be a data scientist in the readwrite group.

In [15]:
#Create data_analyst user
cur2.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
#Assign data analyst to readonly group
cur2.execute("GRANT readonly TO data_analyst;")
#Create data_scientist user
cur2.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
#Assign data scientist to readwrite group
cur2.execute("GRANT readwrite TO data_scientist;")
conn2.commit()

## Testing the Database

The Postgres database has been succesfully set up! Now it is time to test it to ensure everything is configured as intended and that the users have the right permissions.  

I will use the pg_roles tables to inspect privileges related to the database and the information_schema.table_privileges table to inspect table privileges.

In [18]:
#Checking users and groups
cur2.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur2:
    print(user)
print()
#Checking table privileges
cur2.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur2:
    print(user)
conn2.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


The database is now complete! The tests show that the new users cannot create their own tables in the database and that the two groups have the privileges that they are supposed to have.

## Sampling the Data

Here I pull the first 10 results from the boston_crimes table of the crimes schema to sample the data and ensure that it was loaded correctly into the database

In [25]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute('Select * From crimes.boston_crimes')
cur.fetchmany(10)

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101')),
 (6,
  3820,
  'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.29019621'),
  Decimal('-71.07159012')),
 (7,
  724,
  'AUTO THEFT',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.30607218'),
  Decimal('-71.0827326')),
 (8,
  3301,
  'VERBAL DISPUTE',
  datetime.date(2018, 9, 3),
  '